In [3]:
with open('data/t.csv') as file:
    data = file.read().strip()

headers, *data = [line.split(',') for line in data.split('\n')]

In [51]:
add_tag = lambda data, tag: f"<{tag}>{data}</{tag}>"
add_data = lambda data, tag: ''.join([add_tag(d, tag) for d in data])
create_line = lambda data, tag: f"<tr>{add_data(data, tag)}</tr>\n"

html = "<html>\n<body>\n<table>\n"
html += create_line(headers, 'th')
for line in data:
    html += create_line(line, 'td')
html += "</table>\n</body>\n</html>"

print(html)

<html>
<body>
<table>
<tr><th>when</th><th>a</th><th>b</th></tr>
<tr><td>2016-08-12</td><td>1.2</td><td>3</td></tr>
<tr><td>2016-08-13</td><td>3.99003</td><td>4.3</td></tr>
</table>
</body>
</html>


In [4]:
check_header = lambda header: header.replace(' ', '_')
xml_data = lambda data, header: f"<{header}>{data}</{header}>"
xml_line = lambda line: ''.join(xml_data(data, header) for data, header in line)

xml = '<?xml version="1.0"?>\n<file>\n'
xml += f"\t<headers>{','.join(headers)}</headers>\n"
xml += f"\t<data>\n"
for record in data:
    xml += f"\t\t<record>\n"
    headers = [check_header(header) for header in headers]
    xml += f"\t\t\t{xml_line(zip(record, headers))}\n"
    xml += f"\t\t</record>\n"
xml += f"\t</data>\n</file>"

print(xml)

<?xml version="1.0"?>
<file>
	<headers>when,a,b</headers>
	<data>
		<record>
			<when>2016-08-12</when><a>1.2</a><b>3</b>
		</record>
		<record>
			<when>2016-08-13</when><a>3.99003</a><b>4.3</b>
		</record>
	</data>
</file>


In [77]:
json_data = lambda data, header: f'"{header}":"{data}"'
json_line = lambda line: ', '.join(json_data(data, header) for data, header in line)

json = "{\n"
header_str = '", "'.join(headers)
json += f'\t"headers":["{header_str}"],\n'
json += '\t"data":[\n'
for i, record in enumerate(data):
    json += "\t\t{\n"
    json += f"\t\t\t{json_line(zip(record, headers))}\n"
    json += "\t\t}"
    if i != len(data) - 1:
        json += ",\n"
    else:
        json += "\n"
json += "\t]\n}"

print(json)

{
	"headers":["when", "a", "b"],
	"data":[
		{
			"when":"2016-08-12", "a":"1.2", "b":"3"
		},
		{
			"when":"2016-08-13", "a":"3.99003", "b":"4.3"
		}
	]
}


In [8]:
import untangle

with open('output/t.xml','r') as file:
    xmltxt = file.read()

xmldata = untangle.parse(xmltxt)

In [9]:
xmldata = untangle.parse(xmltxt)

headers = xmldata.file.headers.cdata.split(',')
data = {header : [] for header in headers}
for record in xmldata.file.data.record:
    for element in record.children:
        name = element._name.replace('_', " ")
        data[name].append(element.cdata)
data = list(map(list, zip(*data.values())))

add_line = lambda line: ','.join(line) + '\n'

csv = add_line(headers)
for line in data:
    csv += add_line(line)

print(csv)

when,a,b
2016-08-12,1.2,3
2016-08-13,3.99003,4.3



In [130]:
import json

with open('temp.json','r') as file:
    jsontxt = file.read()

data = json.loads(jsontxt)

In [138]:
headers = data['headers']
data = [list(record.values()) for record in data['data']]

add_line = lambda line: ','.join(line) + '\n'

csv = add_line(headers)
for line in data:
    csv += add_line(line)

print(csv)

when,a,b
2016-08-12,1.2,3
2016-08-13,3.99003,4.3

